<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Создание кластера </b> <span style="font-weight:bold; color:green">HDP</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Создание кластера на Azure</a></li>
        <li><a href="#2">Доступ к сервисам кластера</a></li>
        <li><a href="#3">Подготовка к запуску сервера тетрадей</a></li>
        <li><a href="#4">Подключение библиотек и создание контекста</a></li>
        <li><a href="#5">Завершение работы с кластером</a></li>
    </ol>
</div>

<p><b>Исходники для развертывания и остановки кластера Azure</b></p>

In [ ]:
https://github.com/ZEMUSHKA/lsml_hse/blob/master/azure/docs/CREATE_CLUSTER.md

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Создание кластера на Azure</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Вход в Azure</b></p>

<p>Azure CLI 2</p>

In [ ]:
az login

<p>Пример логина</p>

In [ ]:
studentX@zimovnovgmail.onmicrosoft.com

<p>Портал Azure с графическим интерфейсом</p>

In [ ]:
portal.azure.com

<p><b>Создание кластера HDP 2.5 - Hortonworks</b></p>

<p><i>Установка JobLib: </i><a href="https://pythonhosted.org/joblib/installing.html">https://pythonhosted.org/joblib/installing.html</a></p>

In [ ]:
%%bash
python create_cluster.py \
    --user student* \
    --ssh_key ~/.ssh/*.pub \
    --create_shared \
    --create_aux

<p>Отображение публичных IP созданных узлов</p>

In [ ]:
%%bash
az vm list-ip-addresses

<p>Подключение к удаленным узлам</p>

In [ ]:
sudo ssh -i ~/.ssh/YOUR_PRIVATE_KEY ubuntu@VM_PUBLIC_IP

<p>Добавление узлов в файл hosts для разрешение имен</p>

In [ ]:
sudo nano /etc/hosts

In [ ]:
10.0.1.21	cluster1
10.0.1.22	cluster2
10.0.1.23	cluster3

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Доступ к сервисам кластера</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Создание туннеля с помощью команды SSH</p>

<p><b>Способ 1</b> (основной)</p>

In [ ]:
https://docs.microsoft.com/ru-ru/azure/hdinsight/hdinsight-linux-ambari-ssh-tunnel

In [ ]:
sudo ssh -i ~/.ssh/YOUR_PRIVATE_KEY -C2qTnNf -D 8090 ubuntu@VM_PUBLIC_IP_cluster1

<p>Настройка браузера для работы с SOCKS</p>

<p><b>Способ 2</b></p>

In [ ]:
#sudo ssh -i ~/.ssh/YOUR_PRIVATE_KEY -NfL localhost:4050:localhost:8088 ubuntu@VM_PUBLIC_IP_cluster1

<p><b>Способ 3</b></p>

In [ ]:
"""%%bash
az network nsg rule create \
    --resource-group "student1_resources" \
    --nsg-name "security_group" \
    --name "ambari_port" \
    --protocol tcp \
    --direction inbound \
    --priority 1300 \
    --source-address-prefix '*' \
    --source-port-range '*' \
    --destination-address-prefix '*'\
    --destination-port-range 8080 \
    --access allow
"""

<p>Запуск менеджера кластера Ambari</p>

In [ ]:
# в Firefox
http://localhost:8080

<p>Логин и пароль к Ambari</p>

In [ ]:
# login: admin; password: admin

<p>Запустите все службы</p>

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Подготовка к запуску сервера тетрадей</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>На виртуалке cluster1</b></p>

<p>Установленные пакеты python</p>

In [ ]:
pip freeze

<p>Копирование файлов для запуска сервера тетрадей</p>

In [ ]:
sudo chown -R ubuntu /usr/local/backup

In [ ]:
sudo cp -rT /usr/local/backup /home/ubuntu

<p>Создание директории для тетрадей и для текущего занятия</p>

In [ ]:
sudo mkdir -p ~/notebooks/Class_2

<p>Добавление рабочей директории в конфигурационный файл</p>

In [ ]:
sudo nano jupyter_config.py

In [ ]:
c.NotebookApp.notebook_dir = '/home/ubuntu/notebooks'

<p>Запуск менеджера терминалов</p>

In [ ]:
# https://robots.thoughtbot.com/a-tmux-crash-course

In [ ]:
tmux

<p>Разделение окна терминала</p>

In [ ]:
tmux split-window

<p>Установка разрешений запись на директорию в HDFS</p>

In [ ]:
sudo -u hdfs hadoop fs -chmod 777 /user

<p>Запуск сервера тетрадей</p>

In [ ]:
./start_notebook.sh

<p>Доступ к серверу тетрадей через браузер (при использовании SOCKS)</p>

In [ ]:
https://localhost:9999

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Подключение библиотек и создание Spark контекста</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Запустите тетрадь <b>HOWTO_Create_Spark_Context.ipynb </b></p>

In [ ]:
# TODO

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Завершение работы с кластером</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p> 1. Ambari: stop all - остановите все службы</p>

<p> 2. Сохраните тетради и выключите сервер тетрадей</p>

<p>3. python <b>cluster_control.py</b> --user student* --stop</p>

In [ ]:
"""
%%bash
az vm deallocate 
    --name "cluster*" \
    --resource-group "student1_resources"
"""

In [ ]:
"""
%%bash
az vm deallocate \
    --ids $(az vm list --resource-group "student1_resources" --query "[].id" -o tsv)
"""